In [2]:
import tensorflow as tf

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing

import os
import collections
import json
from google.colab import files

In [30]:
class PreisachMemoryLayer(tf.keras.layers.Layer):
    def __init__(self, n_operators, **kwargs):
        super(PreisachMemoryLayer, self).__init__(name="PreisachMemoryLayer", **kwargs)
        self.n_operators = n_operators
        self.first_batch = True
    
    def build(self, input_shape):
        # TODO: check dimensionality of the tensors, most probably they are wrong
        # Change the last dimension of input_shape to be n_operators
        self.input_dim = input_shape
        self.play_operators = np.zeros((input_shape[0], input_shape[1], self.n_operators)) # tf.zeros((input_shape)) # Fix the Data Structure
        self.prev_play_operators = np.zeros((input_shape[0], 1, self.n_operators)) # tf.zeros((input_shape)) # Fix the Data Structure
        self.r = np.zeros(self.n_operators)
        
        for j in range(self.n_operators):
            # Since the input signal is normalized between -1 and 1, these are also min and max
            self.r[j] = (j / self.n_operators) * (1 - (-1))
        
    def call(self, inputs):
        # TODO: Check dimensions of all the data structure, verify if the code is fine for stateful = False
        # TODO: Extend the code to work with stateful = True, using the Play Operators of the previous batch
        # TODO: Check if it is possible to remove the external loop over the batches
        for b in range(self.input_dim[0]):
            for t in range(self.input_dim[1]):
                for j in range(self.n_operators):
                    if t == 0:
                        if self.first_batch:
                            # TODO: FIX the dimensions for the array assignments?
                            self.play_operators[b, t, j] = np.maximum(inputs[b, t] - self.r[j], np.minimum(inputs[b, t] + self.r[j], 0)) # <-- The zero must be a Tensor with proper dimensions!
                        else:
                            self.play_operators[b, t, j] = np.maximum(inputs[b, t] - self.r[j], np.minimum(inputs[b, t] + self.r[j], self.prev_play_operators[b, 0, j]))
                    else:
                        self.play_operators[b, t, j] = np.maximum(inputs[b, t] - self.r[j], np.minimum(inputs[b, t] + self.r[j], self.play_operators[b, t - 1, j]))
                
        if self.first_batch:
            self.first_batch = False
        
        self.prev_play_operators = self.play_operators[:, t, :]
        print(self.prev_play_operators)
        
        # The following comment is used as a reference for the Play Operators definition
        # P_j(t) = max(u(t) - r_j, min(u(t) + r_j, P_j(t - 1)))
        # P_j(0) = max(u(0) - r_j, min(u(0) + r_j, k0))
        # r_j = (j - 1)/n * (max(u(t)) - min(u(t)))  j = 1, ..., n
        
        return self.play_operators

In [ ]:
### DEBUG ###

layer = PreisachMemoryLayer(n_operators=5)
y = layer(np.array([[[0.],[0.]],[[0.],[0.]]]))